# Pytorch Tensor

在 pytorch 這個框架中，所有的運算、資料流動或是其他數學方法的底層都是張量 (tensor)，pytorch tensor 有著和 Numpy 同樣方便的函數以及操作方法，基本上我們可以直接套用 numpy 的邏輯到 tensor 處理上面，除此之外，也有很多類似函數提供使用。

### Build a tensor 

如何建立一個 tensor，又要怎麼把 tensor 移到裝置上面呢? [pytorch tensor](https://pytorch.org/docs/stable/tensors.html)

In [ ]:
import torch # import pytorch library

tensor1 = torch.tensor([[1, 2], [3, 4]])
tensor2 = torch.FloatTensor([[1, 2], [3, 4]])

tensor3 = torch.ByteTensor([[1, 2], [3, 4]]) # You may see the data type in cv2 preprocessing
tensor4 = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32) # Same as tensor1

print(
    f'tensor1: {tensor1}',
    f'\ntensor2: {tensor2}',
    f'\ntensor3: {tensor3}',
    f'\ntensor4: {tensor4}'
    )

tensor1: tensor([[1, 2],
        [3, 4]]) 
tensor2: tensor([[1., 2.],
        [3., 4.]]) 
tensor3: tensor([[1, 2],
        [3, 4]], dtype=torch.uint8) 
tensor4: tensor([[1., 2.],
        [3., 4.]])


## Device

在 tensor 的計算中，我們常常需要非常大量的運算，這些運算之間分別可以以多線程進行，所以一種有效率的方法是把任務分配到不同的 block 和 thread 上。

如果你使用的是 numpy array 物件，那麼你可以用 cupy 套件的提供的 built-in 函數將資料移到 device 上訓練

如果你使用的是 pytorch tensor 物件，pytorch 可以很輕鬆的把所有資料都移到 device  上訓練，但要注意記憶體容量限制

- host: cpu
- device: gpu

如果我想把 tensor 建立在 device 上呢? 

pytorch 支援 cuda 加速，所以我們可以在 cuda 上建立 tensor，或者在 cpu 上建立 tensor 後移到 cuda 上運算

In [ ]:
##### Remember to initialize your GPU setting to prevent RuntimeError alert

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tensor1 = torch.tensor([[1, 2], [3, 4]])
tensor1 = tensor1.to(DEVICE)
#tensor1 = tensor1.cuda()

tensor2 = torch.cuda.FloatTensor([[1, 2], [3, 4]])

tensor3 = torch.ByteTensor([[1, 2], [3, 4]]) # You may see the data type in cv2 preprocessing
tensor4 = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32) # Same as tensor1

print(
    f'DEVICE: {DEVICE}'
    f'\ntensor1: {tensor1}',
    f'\ntensor2: {tensor2}',
    f'\ntensor3: {tensor3}',
    f'\ntensor4: {tensor4}'
    )

#You may see this error occur if the operation of tensors are not on the same divece
try:
    tensor2 + tensor3
except Exception as error: 
    print(error)

DEVICE: cuda
tensor1: tensor([[1, 2],
        [3, 4]], device='cuda:0') 
tensor2: tensor([[1., 2.],
        [3., 4.]], device='cuda:0') 
tensor3: tensor([[1, 2],
        [3, 4]], dtype=torch.uint8) 
tensor4: tensor([[1., 2.],
        [3., 4.]])
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


### Variable

每一個 tensor 都可以看成是"常數"，他們無法進行反向傳播或者運算關係之間的連接，所以我們要用 Variable 方法，將 tensor 註冊到變數中進行梯度的反向傳播

Variable 可以看成是一張圖，這張圖包含了所有被註冊到 Variable 底下的 tensors 以及 tensors 之間的運算關係，因為 Variable 類包含了運算關係，所以我們只要調用 Variable 底下的 ```.backward()``` 就可以進行反向傳播，不需要做其他任何的額外操作

In [ ]:
from torch.autograd import Variable

var =  Variable(tensor4, requires_grad=True) # The default value of 'requires_grad' is False, make sure to set it True !!
tensor5 = torch.mean(var * var)

tensor5.backward() # back propagation

print(
    f'variable:\n {var}'
    f'\ngradient:\n {var.grad}', # The gradient of 1 / n ** 2 * sum(X * X) = 2 / n ** 2 * sum(X) = 2 / 4 * X = 0.5 * X
    f'\ndata:\n {var.data}', # collect your data
    f'\nto numpy:\n {var.data.numpy()}', # collect your data then to numpy
    f'\ndetach tensor:\n {var.detach()}' # detach tensor from training (requires_grad=False)
    )

tensor6 = tensor4.to(DEVICE)
var =  Variable(tensor6, requires_grad=True) # The default value of 'requires_grad' is False, make sure to set it True !!
tensor5 = torch.mean(var * var)

tensor5.backward() # back propagation

try:
    var.data.numpy()
except Exception as error:
    print('\n\n', error)
    print(f'move to host memory: {var.data.cpu().numpy()}')

variable:
 tensor([[1., 2.],
        [3., 4.]], requires_grad=True)
gradient:
 tensor([[0.5000, 1.0000],
        [1.5000, 2.0000]]) 
data:
 tensor([[1., 2.],
        [3., 4.]]) 
to numpy:
 [[1. 2.]
 [3. 4.]] 
detach tensor:
 tensor([[1., 2.],
        [3., 4.]])


 can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
move to host memory: [[1. 2.]
 [3. 4.]]


### Define your constant and variable in Neural Network

在 torch.nn.Module 用來定義常量與變數的方法是用 ```nn.Parameter()``` 而不是 ```Variable```

注意: Variable 並不會放到網路中，所以在 optimizer 更新時不會更新 Variable 參數

- ```nn.Parameter():``` 將參數放在網路中，同時設定 ```requires_grad = True```
- ```register_buffer():``` 在網路中設定常量，並不會在 ```step()``` 方法中更新參數
- ```register_parameter():``` 在網路中設定變量，會在 ```step()``` 方法中更新參數

In [ ]:
import torch.nn as nn

class MyModule(nn.Module):

    def __init__(self):
        super().__init__()

        self.variable = Variable(torch.Tensor([5]))
        self.parameter = nn.Parameter(torch.Tensor([10]))
        self.register_buffer('buffer', torch.Tensor([15]))
        self.register_parameter('register_param', self.parameter)

    def forward(self, x):

        print('\n\nbuffer:\n', self.buffer)
        print('\n', self.parameter)
        print('\nvariable:\n', self.variable)

        out = x + self.parameter + self.parameter + self.buffer
        out.backward()

        print('\ngradient for parameter:\n', self.parameter.grad)
        print('\ngradient for buffer:\n', self.buffer.grad)
        print('\nIf parameter require gradient:\n', self.register_param.requires_grad)
        print('\nIf buffer require gradient:\n', self.buffer.requires_grad)

        return out

net = MyModule()
for param in net.parameters():
    print(param)
out = net.forward(1)
out

Parameter containing:
tensor([10.], requires_grad=True)


buffer:
 tensor([15.])

 Parameter containing:
tensor([10.], requires_grad=True)

variable:
 tensor([5.])

gradient for parameter:
 tensor([2.])

gradient for buffer:
 None

If parameter require gradient:
 True

If buffer require gradient:
 False


tensor([36.], grad_fn=<AddBackward0>)

## Tensor operation

這裡重點介紹一些重要的 tensor 運算函數

- ```torch.cat():``` 將一個 list 的 tensor 進行串接
- ```x.view():``` 將 tensor 的維度重建，同 reshape
- ```x.permute():``` 將 tensor 的維度做調換 

In [ ]:
x = torch.Tensor([[1, 2], [3, 4]])
y = torch.Tensor([[5, 6], [7, 8]])

print('concatenate:\n', torch.cat([x, y], axis=1))
print('reshape:\n', x.view(1, 4))
print('permute:\n', x.permute(1, 0))